# Graph Neural Network

Author: Adam Darmanin

## Paper

[Kika, Alda, et al. "Imbalance Node Classification with Graph Neural Networks (GNN): A Study on a Twitter Dataset."](https://www.proquest.com/openview/707deabdf2dee201896409a9a4fccfb7/1?pq-origsite=gscholar&cbl=5444811)

In [3]:
import os
import numpy as np
import tensorflow as tf
from spektral.layers import GATConv
from py2neo import Graph as NeoGraph
import spacy
from dotenv import load_dotenv

load_dotenv()

nlp = spacy.load("en_core_web_sm")


def get_spacy_embedding(word):
    return nlp(word).vector


client_id = os.getenv("N4J_USER")
client_secret = os.getenv("N4J_PW")
neo4j_graph = NeoGraph("bolt://localhost:7687", auth=(client_id, client_secret))
node_query = "MATCH (n:Subreddit) RETURN n.name as name"
node_data = neo4j_graph.run(node_query).data()
word_query = "MATCH (n:Word) RETURN n.name as name"
word_data = neo4j_graph.run(word_query).data()

node_features = np.array(
    [get_spacy_embedding(node["name"]) for node in node_data + word_data]
)

num_nodes = len(node_data) + len(word_data)
adjacency_matrix = np.zeros((num_nodes, num_nodes))
edge_query = "MATCH (n:Subreddit)-[r]->(m:Word) RETURN id(n) as source, id(m) as target"
edge_data = neo4j_graph.run(edge_query).data()
for edge in edge_data:
    source = edge["source"]
    target = edge["target"]
    adjacency_matrix[source, target] = 1

edge_features = np.ones((len(edge_data), 1))
node_features_tf = tf.convert_to_tensor(node_features, dtype=tf.float32)
adjacency_matrix_tf = tf.convert_to_tensor(adjacency_matrix, dtype=tf.float32)
edge_features_tf = tf.convert_to_tensor(edge_features, dtype=tf.float32)


class GNNModel(tf.keras.Model):
    def __init__(self, n_out):
        super().__init__()
        self.conv1 = GATConv(32, activation="relu")
        self.conv2 = GATConv(n_out)

    def call(self, inputs):
        x, a, e = inputs
        x = self.conv1([x, a, e])
        x = self.conv2([x, a, e])
        return x


number_of_classes = 2
model = GNNModel(number_of_classes)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
# Adam: the DIMs must be the same here
model.fit(
    x=[node_features_tf, adjacency_matrix_tf, edge_features_tf], epochs=30, batch_size=1
)

Epoch 1/30


c:\Users\adamd\.conda\envs\tf\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


ValueError: in user code:

    File "c:\Users\adamd\.conda\envs\tf\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\adamd\.conda\envs\tf\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\adamd\.conda\envs\tf\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\adamd\.conda\envs\tf\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\adamd\.conda\envs\tf\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\adamd\AppData\Local\Temp\__autograph_generated_filel93fmm0a.py", line 11, in tf__call
        x = ag__.converted_call(ag__.ld(self